## Вычислить $ \sqrt[k]{a} $ 

In [1]:
import numpy as np

def printable_test(a, k, f, prc=1e-4):
    ans = f(a, k)
    print(f'Our result: {a}^(1/{k}) ~ {ans:.10f}')
    print(f'True result: {a**(1/k):.10f}\n')
    print(f'Approx a ~ {ans**k:.10f}')
    print(f'True a = {a}')
    assert abs(a - ans**k) < prc, f'the answer differs by {abs(a - ans**k):.10f} from the true one'
    
def not_printable_test(a, k, f, prc=1e-4):
    ans = f(a, k)
    assert abs(a - ans**k) < prc, f'f({a}, {k}): the answer differs by {abs(a - ans**k):.10f} from the true one'
    
def test(func):
    rng = np.random.default_rng(12345)
    test_len = 1000
    vals = rng.integers(low=0, high=1000, size=test_len)
    pws = rng.integers(low=1, high=100, size=test_len)

    for a, k in zip(vals, pws):
        not_printable_test(a, k, func)
        
    print(f'All {test_len} tests have passed!')


In [2]:
def root_bisection(a: float, k: float, iters=1000) -> float:
    def f(x):
        return x**k - a

    assert k > 0, 'Negative `k` values are not allowed'

    l, r = 0, a
    for _ in range(iters):
        m = l + (r - l) / 2
        if f(m) * f(l) <= 0:
            r = m
        else:
            l = m
    return l + (r - l) / 2


In [3]:
test(root_bisection)

All 1000 tests have passed!


In [4]:
printable_test(1350, 12, root_bisection)
print('\n')
printable_test(-1, 1, root_bisection)

Our result: 1350^(1/12) ~ 1.8233126596
True result: 1.8233126596

Approx a ~ 1350.0000000000
True a = 1350


Our result: -1^(1/1) ~ -1.0000000000
True result: -1.0000000000

Approx a ~ -1.0000000000
True a = -1


In [5]:
def root_newton(a: float, k: float, iters=1000) -> float:
    def f(x):
        return x**k - a

    def dx(x):
        return k * x**(k - 1)

    assert k > 0, 'Negative `k` values are not allowed'
    
    x = 1
    for _ in range(iters):
        x = x - f(x) / dx(x)

    return x


In [6]:
test(root_newton)

All 1000 tests have passed!


In [7]:
printable_test(1350, 12, root_newton)
print('\n')
printable_test(-1, 1, root_newton)

Our result: 1350^(1/12) ~ 1.8233126596
True result: 1.8233126596

Approx a ~ 1350.0000000000
True a = 1350


Our result: -1^(1/1) ~ -1.0000000000
True result: -1.0000000000

Approx a ~ -1.0000000000
True a = -1


## Дан многочлен P степени не больше 5 и отрезок [L, R]

Локализовать корни: $ P(L_i) \cdot P(R_i) <0 $ 

И найти на каждом таком отрезке корни

In [8]:
from typing import List
import numpy as np

class Polynom:
    def __init__(self, coefs: List[float]):
        # self.coefs = [a0, a1, a2, ...., an]
        self.coefs = coefs

    def __str__(self):
        if not self.coefs:
            return ''
        descr = str(self.coefs[0])
        for i, coef in enumerate(self.coefs[1:]):
            sign = '+' if coef > 0 else '-'
            descr += f' {sign} {abs(coef)} * x^{i + 1}'
        return descr
    
    def __repr__(self):
        return self.__str__()

    def value_at(self, x: float) -> float:
        res = 0
        for i, coef in enumerate(self.coefs):
            res += x**i * coef
        return res
    
    def dx(self):
        if not self.coefs or len(self.coefs) == 1:
            return Polynom([0])
        return Polynom([(i + 1) * coef for i, coef in enumerate(self.coefs[1:])])
    
    def is_root(self, x: float, prc=1e-4) -> bool:
        return abs(0 - self.value_at(x)) < prc
    
    def root_segments(self, l: float, r: float, min_seg_len=1e-2) -> List[List[float]]:
        segs = []
        prev_end, cur_end = l, l + min_seg_len

        while cur_end < r:
            if self.value_at(prev_end) * self.value_at(cur_end) < 0:
                segs.append([prev_end, cur_end])
                prev_end = cur_end

            if self.value_at(cur_end) == 0:
                move = min_seg_len / 10
                segs.append([prev_end, cur_end + move])
                prev_end = cur_end + move

            cur_end += min_seg_len

        return segs
    
    def find_single_root(self, l: float, r: float, iters=1000) -> float:
        for _ in range(iters):
            m = l + (r - l) / 2
            if self.value_at(l) * self.value_at(m) < 0:
                r = m
            else:
                l = m
        return l + (r - l) / 2
    
    def find_roots(self, l: float, r: float) -> List[float]:
        roots = []
        segs = self.root_segments(l, r)
        for seg_l, seg_r in segs:
            roots.append(self.find_single_root(seg_l, seg_r))
        
        return roots
    
    def check_roots(self, roots: List[float]) -> bool:
        return np.all([self.is_root(x) for x in roots])
    
    def find_min(self, l: float, r: float) -> float:
        assert self.coefs, 'Polynom must contain at least one coef'

        if len(self.coefs) == 1:
            return self.coefs[0]

        pts = [l, *self.dx().find_roots(l, r), r]
        return min(self.value_at(pt) for pt in pts)


In [9]:
# x^3 + 97.93 x^2 - 229.209 x + 132.304
# (x - 1.1) * (x - 1.2) * (x + 100.23)
p = Polynom([132.304, -229.209, 97.93, 1])
p.find_roots(-1000, 1000)

[-100.23000003891843, 1.1000394905623918, 1.1999605483560587]

In [10]:
p.find_min(-10, 10)

-0.25305001541232075

In [11]:
Polynom([1, 2, 1]).find_roots(-1000, 1000)

[-0.9990000007645006]

In [12]:
Polynom([1, -2]).find_roots(-1000, 1000)

[0.5000000000000004]

## Найти минимум функции $ e^{ax} + e^{-bx} + c(x - d)^2$

In [13]:
from numpy import exp
from typing import Tuple

class ExpMinFinder:
    def __init__(self, a: float, b: float, c: float, d: float):
        if a <= 0 or b <= 0 or c <= 0:
            raise ValueError("Parameters must be non-negative")
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        
    def f(self, x) -> float:
        return exp(self.a * x) + exp(-self.b * x) + self.c * (x - self.d)**2
    
    def dx(self, x) -> float:
        return self.a * exp(self.a * x) - self.b * exp(-self.b * x) + 2 * self.c * (x - self.d)
    
    def ddx(self, x) -> float:
        return self.a**2 * exp(self.a * x) + self.b**2 * exp(-self.b * x) + 2 * self.c
    
    def min_bisection(self, iters=1000) -> Tuple[float, float]:
        l, r = -100, 100
        for _ in range(iters):
            m = l + (r - l) / 2
            if self.dx(m) * self.dx(l) < 0:
                r = m
            else:
                l = m

        min_at = l + (r - l) / 2
        return min_at, self.f(min_at)
    
    def min_newton(self, iters=1000) -> Tuple[float, float]:
        x = 1
        for _ in range(iters):
            x = x - self.dx(x) / self.ddx(x)

        return x, self.f(x)

    def min_ternary(self, iters=1000) -> Tuple[float, float]:
        l, r = -100, 100
    
        for _ in range(iters):
            m1 = l + (r - l) / 3
            m2 = r - (r - l) / 3
            if self.f(m1) >= self.f(m2):
                l = m1
            else:
                r = m2
        min_at = l + (r - l) / 2

        return min_at, self.f(min_at)


In [14]:
def test_exp():
    rng = np.random.default_rng(12345)
    test_len = 100
    a_ = rng.integers(low=1, high=10, size=test_len)
    b_ = rng.integers(low=1, high=10, size=test_len)
    c_ = rng.integers(low=1, high=10, size=test_len)
    d_ = rng.integers(low=1, high=10, size=test_len)
    
    
    for a, b, c, d in zip(a_, b_, c_, d_):
        m = ExpMinFinder(a, b, c, d)
        
        assert abs(m.min_bisection()[1] - m.min_newton()[1]) < 1e-3, \
            f'Results: {m.min_bisection():.3f} {m.min_newton():.3f}, values: {a, b, c, d}'
        assert abs(m.min_newton()[1] - m.min_ternary()[1]) < 1e-3, \
            f'Results: {m.min_newton():.3f} {m.min_ternary():.3f}, values: {a, b, c, d}'
    
    print(f'All {test_len} tests have passed')
    
test_exp()

/var/folders/n7/k2zyw7490b97hfmqkg39c6080000gn/T/ipykernel_79092/1883086689.py:17: RuntimeWarning: overflow encountered in exp
  return self.a * exp(self.a * x) - self.b * exp(-self.b * x) + 2 * self.c * (x - self.d)


All 100 tests have passed


In [15]:
m = ExpMinFinder(1, 2, 3, 4)

In [16]:
m.min_bisection()

(2.315283729985808, 18.652352027148623)

In [17]:
m.min_newton()

(2.315283729985808, 18.652352027148623)

In [18]:
m.min_ternary()

(2.315283755759233, 18.65235202714863)